*Moved CleanText_Arabic function to preprocessing.ipynb*

In [20]:
import pandas as pd
import string

from sklearn.base import BaseEstimator, TransformerMixin

from nltk.corpus import stopwords
from textblob import TextBlob
import re
from snowballstemmer import stemmer
ar_stemmer = stemmer("arabic")

In [21]:
training_path = "data/raw/2018-Valence-oc-Ar-train.txt"
dev_path = "data/raw/2018-Valence-oc-Ar-dev.txt"
test_path = "data/raw/2018-Valence-oc-Ar-test.txt"

training_outpath = "data/cleaned/Ar_cleaned_training.txt"
dev_outpath = "data/cleaned/Ar_cleaned_dev.txt"
test_outpath = "data/cleaned/Ar_cleaned_test.txt"

stopwords_path = "utilities/arabic-stop-words-list.txt"

In [22]:
training_df = pd.read_table(training_path)
dev_df = pd.read_table(dev_path)
test_df = pd.read_table(test_path)

training_df

,ID,Tweet,Affect Dimension,Intensity Class
0,2018-Ar-01961,إلىٰ متىٰ الألم يغلب على الفرح,valence,-3: very negative emotional state can be inferred
1,2018-Ar-03289,@Al3mriRami @Holyliviuss كل مافي الأمر أني غاض...,valence,-2: moderately negative emotional state can be...
2,2018-Ar-04349,يحذركم ويخوفكم من نفسه اذا ارتكبتم ذنب او معصي...,valence,-2: moderately negative emotional state can be...
3,2018-Ar-03640,💞 💞 صباحكم سعادة في اليوم المبارك تقبل الله صي...,valence,3: very positive emotional state can be inferred
4,2018-Ar-01176,@sjalmulla شفته قبل اسبوع ومتشوقه عليه وايد ال...,valence,2: moderately positive emotional state can be ...
...,...,...,...,...
927,2018-Ar-01836,@GafnZx 😳\n\nهذا بالحلال يابنت الحلال👌🏻😀🎉,valence,1: slightly positive emotional state can be in...
928,2018-Ar-03475,الصادم في حقيقة الامر ان خلود رجعت مشاكسة مثل ...,valence,-2: moderately negative emotional state can be...
929,2018-Ar-01490,كل تلك الأكتاف قادرة على حمل رأسك لكن و ل سوء ...,valence,-1: slightly negative emotional state can be i...
930,2018-Ar-01710,بسم الله الرحمن الرحيم اعوذ بالله ماحسيت بطعم ...,valence,-2: moderately negative emotional state can be...


In [23]:
with open(stopwords_path, "r", encoding="utf-8") as infile: 
    stopwords = list()
    for line in infile:
        line = line.replace("\n", "")
        stopwords.append(line)

In [24]:
def adapt_valence_scores(df): 
    valence_list = list()
    for index, row in df.iterrows():
        valence = row["Intensity Class"]
        valence = valence.replace(valence, valence[:2].replace(":", ""))
        valence_list.append(valence)
    return valence_list
        
training_valence = adapt_valence_scores(training_df)
dev_valence = adapt_valence_scores(dev_df)
test_valence = adapt_valence_scores(test_df)

In [25]:
class CleanText_Arabic(BaseEstimator, TransformerMixin):
    """
    From https://towardsdatascience.com/sentiment-analysis-with-text-mining-13dd2b33de27
    """
    def remove_repeating_char(self, input_text):
        return re.sub(r'(.)\1+', r'\1\1', input_text) #keep 2 repeat
    
    def remove_mentions(self, input_text):
        return re.sub(r'@\w+', '', input_text)
    
    def remove_urls(self, input_text):
        return re.sub(r"http\S+ | www\S+" , "لينك", input_text)
    
    def remove_hashtags(self, input_text):
        return re.sub(r"#", "", input_text)
    
    def emoji_oneword(self, input_text):
        # By compressing the underscore, the emoji is kept as one word
        return input_text.replace('_','')
    
    def remove_punctuation(self, input_text):
        # Make translation table
        punct = string.punctuation
        trantab = str.maketrans(punct, len(punct)*' ')  # Every punctuation symbol will be replaced by a space
        return input_text.translate(trantab)
    
    def remove_digits(self, input_text):
        return re.sub('\d+', '', input_text)
    
    def remove_stopwords(self, input_text):
        # Some words which might indicate a certain sentiment are kept via a whitelist
        ### whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords) and len(word) > 1] 
        return " ".join(clean_words)
    
    #def stem(self, input_text):
        #words = input_text.split()
        #stemmed_words = [ar_stemmer.stemWord(word) for word in words]
        #return " ".join(stemmed_words)
    
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X, **transform_params):
        clean_X = X.apply(self.remove_hashtags).apply(self.remove_repeating_char).apply(self.remove_mentions).apply(self.remove_urls).apply(self.emoji_oneword).apply(self.remove_punctuation).apply(self.remove_digits).apply(self.remove_stopwords)
        return clean_X

In [26]:
ct = CleanText_Arabic()
training_clean = ct.fit_transform(training_df.Tweet)
dev_clean = ct.fit_transform(dev_df.Tweet)
test_clean = ct.fit_transform(test_df.Tweet)

In [27]:
training_clean

0                             إلىٰ متىٰ الألم يغلب الفرح
1      مافي الأمر أني غاضب أننا نظهر نقدم أساطيرنا بأ...
2      يحذركم ويخوفكم ارتكبتم ذنب معصيه سينزل بكم عقا...
3               صباحكم سعادة المبارك تقبل صيامنا وقيامنا
4      شفته اسبوع ومتشوقه وايد الصراحه 😍😍 بالتوفيق يا...
                             ...                        
927                        nهذا بالحلال يابنت الحلال👌🏻😀🎉
928    الصادم حقيقة الامر خلود رجعت مشاكسة وتضايقني م...
929         الأكتاف قادرة حمل رأسك سوء حظك الكتف تحب وجع
930    بسم الرحمن الرحيم اعوذ بالله ماحسيت بطعم الرعب...
931    بتخاف اخواتها اوى دايما سر اخوها بتكون فرحة ال...
Name: Tweet, Length: 932, dtype: object

In [28]:
training_cleaned = pd.DataFrame(training_clean)
training_cleaned['Valence score']= training_valence

dev_cleaned = pd.DataFrame(dev_clean)
dev_cleaned['Valence score']= dev_valence

test_cleaned = pd.DataFrame(test_clean)
test_cleaned['Valence score']= test_valence

In [ ]:
training_cleaned.to_csv(training_outpath, sep="\t")
dev_cleaned.to_csv(dev_outpath, sep="\t")
test_cleaned.to_csv(test_outpath, sep="\t")

In [ ]:
training_cleaned